In [1]:
__author__ = 'Erwin Chen'
%matplotlib inline
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import style
from pylab import MaxNLocator
from astropy.io import fits
from astropy.table import Table, join, Column
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets

style.use('fivethirtyeight')

In [2]:
ap_file = fits.open('allStarCannon-l31c.2.fits')
ap_data = ap_file[1].data
ap_header = ap_file[1].header
# print ap_header

In [3]:
## load data from APOGEE
ap_file = fits.open('allStarCannon-l31c.2.fits')
# print (ap_file[1].header.keys)
ap_data = ap_file[1].data
feature_names = ['APOGEE_ID', 'FE_H', 'C_H', 'CI_H', 'N_H', 'O_H', 'NA_H', 'MG_H', 'AL_H', 'SI_H', 'P_H', 'S_H', 'K_H', 'CA_H', 'TI_H', 'TIII_H', 'V_H', 'CR_H', 'MN_H', 'CO_H', 'NI_H']
feature_names = np.array(feature_names)
element_names = ['FE_H', 'C_H', 'CI_H', 'N_H', 'O_H', 'NA_H', 'MG_H', 'AL_H', 'SI_H', 'P_H', 'S_H', 'K_H', 'CA_H', 'TI_H', 'TIII_H', 'V_H', 'CR_H', 'MN_H', 'CO_H', 'NI_H']
element_names = np.array(element_names)
elements = np.array([name.replace('_H', '').title() for name in element_names])
print("The following elements are in the data:")
print(elements)

## append data into columns
ap_cols = []
for name in feature_names:
    ap_cols.append(ap_data.field(name))
ap_cols = np.array(ap_cols)
ap_cols = ap_cols.T

## create a table with the data columns
dtype = ['float' for n in range(len(feature_names))]
dtype[0] = 'string'
ap_table = Table(data=ap_cols, names=feature_names, dtype=dtype)

ap_file.close()

The following elements are in the data:
['Fe' 'C' 'Ci' 'N' 'O' 'Na' 'Mg' 'Al' 'Si' 'P' 'S' 'K' 'Ca' 'Ti' 'Tiii' 'V'
 'Cr' 'Mn' 'Co' 'Ni']


In [4]:
## add membership and number labels for clusters
known_clusters = np.loadtxt('table4.dat', usecols=(0, 1), dtype=('S', 'S'), unpack=True)
member_IDs = known_clusters[0]
member_names = known_clusters[1]
labels = np.zeros(len(member_IDs))-1
cluster_names = list(set(member_names))

names = ['APOGEE_ID', 'cluster_name']
dtype=['string', 'string']
member_table = Table(data=[member_IDs, member_names], names=names, dtype=dtype)
ap_table = join(ap_table, member_table, keys='APOGEE_ID', join_type='left')

In [5]:
## fill missing values
ap_table['cluster_name'].fill_value = 'Background'
for element in element_names:
    ap_table[element].mask = np.isnan(ap_table[element])
    ap_table[element].fill_value = -9999.
ap_table = ap_table.filled()

In [6]:
## group table by cluster names

ap_table_by_name = ap_table.group_by('cluster_name')
cluster_names = [name for name in ap_table_by_name.groups.keys['cluster_name'] if name != 'Background']

In [7]:
## show clusters

for group in ap_table_by_name.groups:
    name = group['cluster_name'][0]
    print("%s has %i members"%(name, len(group)))

Background has 276550 members
M107 has 18 members
M13 has 87 members
M15 has 18 members
M2 has 40 members
M3 has 140 members
M35 has 1 members
M5 has 203 members
M53 has 19 members
M67 has 38 members
M71 has 14 members
M92 has 50 members
N188 has 10 members
N2158 has 10 members
N2420 has 9 members
N4147 has 3 members
N5466 has 11 members
N6791 has 23 members
N6819 has 30 members
N7789 has 5 members
Pleiades has 92 members


In [8]:
## functions for getting chemical abundances and their labels

def get_element(group, element):
    if element == 'FE_H':
        return group[element]
    else:
        return group[element] - group['FE_H']
    
def get_label(element):
    label = element.replace('_H', '').title()
    if element == 'FE_H':
        return '[' + label + '/H]'
    else:
        return '[' + label + '/Fe]'

In [9]:
## individual histogram

def ind_hist(view_cluster, view_element):
    
    mask = ap_table_by_name.groups.keys['cluster_name'] == view_cluster
    group = ap_table_by_name.groups[mask]
    valid = np.where(group[view_element] > -999.)[0]

    xlabel = get_label(view_element)
    ylabel = "Number of Members"
    X = get_element(group, view_element)
    X = X[valid]

    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6,10))

    y_axis = ax[0].get_yaxis()
    y_axis.set_major_locator(MaxNLocator(integer=True))

    ax[0].hist(X, histtype='bar', ec='black', label=group['cluster_name'][0])
    ax[0].set_xlabel(xlabel)
    ax[0].set_ylabel(ylabel)
    ax[0].legend(loc=0)
    y_axis = ax[0].get_yaxis()
    y_axis.set_major_locator(MaxNLocator(integer=True))

    ax[1].hist(X, histtype='bar', ec='black', cumulative=True, label=group['cluster_name'][0])
    ax[1].set_xlabel(xlabel)
    ax[1].set_ylabel(ylabel)
    ax[1].legend(loc=0)
    y_axis = ax[1].get_yaxis()
    y_axis.set_major_locator(MaxNLocator(integer=True))

    plt.show()

In [10]:
interact(ind_hist, view_cluster=cluster_names, view_element=element_names)

<function __main__.ind_hist>

In [11]:
## 2d_scatter
def two_element_scatter(element_0, element_1, view_cluster):
    # colors
    num_cluster = len(view_cluster)
    colors = np.linspace(0, 1, num_cluster)
    cm = plt.get_cmap('rainbow')

    fig, ax = plt.subplots(figsize=(8,6))

    i = 0
    for name in view_cluster:
        mask = ap_table_by_name.groups.keys['cluster_name'] == name
        group = ap_table_by_name.groups[mask]
        valid = np.where((group[element_0] > -999.) * (group[element_1] > -999.))[0]
        if name != 'Background':
            X = get_element(group, element_0)
            Y = get_element(group, element_1)
            X = X[valid]
            Y = Y[valid]
            ax.plot(X, Y, '.', c=cm(colors[i]), label=name)
            i += 1

    xlabel = get_label(element_0)
    ylabel =get_label(element_1)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend(loc=0, fontsize='x-small', ncol=5)
    plt.show()

In [13]:
interact(two_element_scatter, element_0 = element_names, element_1 = element_names, 
         view_cluster = widgets.SelectMultiple(options=cluster_names, value=['M107'], description='cluster name'))

Select one or more clusters and specify two elements


<function __main__.two_element_scatter>